<a href="https://colab.research.google.com/github/russellemergentai/MistralDocker/blob/main/TaskAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

T4 is fine.

In [1]:
#login

from google.colab import drive
drive.mount('/content/drive')

from huggingface_hub import login
from google.colab import userdata

# Load the secret token
HF_TOKEN = userdata.get('HF_TOKEN')

# Check if the token is set and log in
if HF_TOKEN:
    login(HF_TOKEN)
    print("logged in to Hugging Face")
else:
    print("Hugging Face login failed")


# init LLM
!pip install -U bitsandbytes
!pip install torch transformers langchain
!pip install -U langchain-community
!pip install faiss-gpu-cu12

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

# load model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

#model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V5"
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side='left', add_eos_token=True, use_fast=False)

# create retriever
from langchain.document_loaders import TextLoader
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, SequentialChain
from google.colab import output
from langchain import PromptTemplate

import os
from pathlib import Path

def create_directory_retriever(directory_path):
    all_documents = []

    for file_path in Path(directory_path).rglob('*'):
        if file_path.is_file():
            loader = TextLoader(str(file_path), encoding='UTF-8')
            documents = loader.load()
            all_documents.extend(documents)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=150, chunk_overlap=75, separators=[" ", ",", "\n"]
    )

    texts = text_splitter.split_documents(all_documents)

    model_path = "intfloat/e5-large-unsupervised"

    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device': 'cuda'},
        encode_kwargs={'normalize_embeddings': False}
    )

    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever()

    return retriever


# Define common pipeline parameters
common_params = {
    "torch_dtype": torch.bfloat16,
    "trust_remote_code": True,
    "device_map": {"": 0},
    "num_return_sequences": 1,
    "repetition_penalty": 1.5
}



# format output
def print_sections(result):
  words = result.split()
  for i in range(0, len(words), 12):
      chunk = words[i:i + 12]
      print(" ".join(chunk) + "\n")




# retrieve from data directory
def retrieval_query_data():

  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request. Your response style should be very succinct.
  {query}
  [/INST]
  </s>
   <s>
  """

  path="/content/drive/MyDrive/Target"

  retriever = create_directory_retriever(path)
  output.clear()

  query = input("enter your query on the uploaded data: ")

  common_params = {
    'max_length': 512,
    'eos_token_id': tokenizer.eos_token_id,
  }

  # Create the pipeline for text generation with output length constraint
  pipelineQuery = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      **common_params,
      max_new_tokens=512
  )

  llmPipelineQuery = HuggingFacePipeline(pipeline=pipelineQuery, model_kwargs={"temperature": 0.1})
  qa = RetrievalQA.from_chain_type(llm=llmPipelineQuery, retriever=retriever, return_source_documents=False)
  result = qa.run({"query": query})

  del pipelineQuery
  del llmPipelineQuery
  del qa
  del retriever
  import gc
  gc.collect()

  #print_sections(result)
  print(result)



# query the base model with a commentary from the fine tuned model
def standard_query():
  prompt_template="""
  <s>
  [INST]
  Below is an instruction that describes a task. Write a response that appropriately completes the request. Keep your response brief.
  {query}
  [/INST]
  </s>
   <s>
  """

  query = input("enter the llm query: ")

  # Initialize pipelines with specific parameters
  pipelineBase = pipeline(
      "text-generation",
      model=llm,
      tokenizer=tokenizer,
      #max_length=500,
      #do_sample=True,
      eos_token_id=tokenizer.eos_token_id,
      **common_params,
      max_new_tokens=200
  )

  # Create HuggingFacePipeline objects
  llmPipeline = HuggingFacePipeline(pipeline=pipelineBase, model_kwargs={"temperature": 0.1})
  PROMPT = PromptTemplate.from_template(prompt_template)
  llm_chain = PROMPT | llmPipeline
  result = llm_chain.invoke({"query": query})

  del pipelineBase
  del llmPipeline
  del PROMPT
  del llm_chain
  import gc
  gc.collect()

  print_sections(result)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
logged in to Hugging Face
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.29
    Uninstalling langchain-core-0.3.29:
      Successfully uninstalled langchain-core-0.3.29
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.0 MB/s eta 0:00:00


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [2]:
# main loop
def main():
    functions = {'1': standard_query, '2': retrieval_query_data}

    while True:
        user_input = input("Enter 1 llm query, 2 data retrieval query or 'x' to exit: ").lower()
        output.clear()

        if user_input in functions:
            functions[user_input]()
        elif user_input == 'x':
            print("Exiting.")
            break
        else:
            print("Invalid input, please try again.")

if __name__ == "__main__":
    main()

Exiting.
